# CSS Colour Converter

In [23]:
import os
os.chdir('computer-systems/bits-and-bytes/css-colour-convert/')

The goal with this problem is convert hex colours in CSS to RGB form. For example:

| Type | Hex | RGB |
| --- | --- | --- |
| Normal | `#00FF00` | `rgb(0, 255, 0)` |
| Alpha | `#0000FFC0` | `rgb(0 0 255 / 0.75294)` |
| Short | `#123` | `rgb(17 34 51)` |
| Short alpha | `#00f8` | `rgb(0 255 / 0.53333)` |

Let's start by dealing with the normal six-digit hex case.

In [61]:
def hex_to_rgb(hex: int) -> str:
    r = (hex & 0xff0000) >> 16
    g = (hex & 0x00ff00) >> 8
    b = hex & 0x0000ff

    return f"rgb({r} {g} {b})"

cases = [(0xff0000, "rgb(255 0 0)"), (0x00ff00, "rgb(0 255 0)"), (0x0000ff, "rgb(0 0 255)")]
for case in cases:
    got = hex_to_rgb(case[0])
    want = case[1]
    assert got == want, f"{got} vs {want}"

The next step is processing all the input. The simplest way I can think of doing this is with a regex.

In [33]:
import re

def helper(match):
    return hex_to_rgb(int(match.group()[1:], 16))

def run(path):
    # TODO: Just iterate over regex matches?
    res = []
    with open(path) as f:
        for l in f.readlines():
            l = re.sub("#\w{6}", helper, l)
            res.append(l)
    return "".join(res)

In [37]:
print(run("simple.css"))

/* 🎄🎄🎄 */

body {
    color: rgb(254 3 10);
    background-color: rgb(15 13 239);
}



In [27]:
with open("simple_expected.css") as f:
    print(f.read())

/* 🎄🎄🎄 */

body {
    color: rgb(254 3 10);
    background-color: rgb(15 13 239);
}



In [47]:
import difflib

got = run("simple.css")
with open("simple_expected.css") as f:
    want = f.read()

diff = '\n'.join(difflib.unified_diff(got.splitlines(), want.splitlines()))
print(diff if diff != "" else "pass")

pass


Now for the advanced case. The tricky part is that there are a few different formats:
```
#RGB        // The three-value syntax
#RGBA       // The four-value syntax
#RRGGBB     // The six-value syntax
#RRGGBBAA   // The eight-value syntax
```

According to MDN, the rule is that if you have just one colour, it should be duplicated, e.g., `#123` becomes `#112233`.

In [147]:
def canonicalise(raw: str) -> str:
    match len(raw):
        case 3: return f"{raw[0]}{raw[0]}{raw[1]}{raw[1]}{raw[2]}{raw[2]}00"
        case 4: return f"{raw[0]}{raw[0]}{raw[1]}{raw[1]}{raw[2]}{raw[2]}{raw[3]}{raw[3]}"
        case 6: return f"{raw}00"
        case 8: return raw
        case _: raise Exception("invalid hex string")
         
def parse_hex(hex: str) -> int:
    return int(hex, 16)

def hex_to_rgb(hex: int) -> str:
    r = (hex & 0xff000000) >> 24
    g = (hex & 0x00ff0000) >> 16
    b = (hex & 0x0000ff00) >> 8
    a = (hex & 0x000000ff)

    if a > 0:
        return f"rgba({r} {g} {b} / {a / 255:.5f})"

    return f"rgb({r} {g} {b})"

cases = [
    ("0000FFC0", "rgba(0 0 255 / 0.75294)"),
    ("ff00ff", "rgb(255 0 255)"),
    ("123", "rgb(17 34 51)"),
    ("00f8", "rgba(0 0 255 / 0.53333)")
]
for case in cases:
    got = hex_to_rgb(parse_hex(canonicalise(case[0])))
    want = case[1]
    assert got == want, f"{got} vs {want}"

In [148]:
pattern = "#([a-fA-F0-9]{3,8})"

def helper(match):
    return hex_to_rgb(parse_hex(canonicalise(match.group(1))))

def run(path):
    with open(path) as f:
        return re.sub(pattern, helper, f.read())

In [149]:
print(run("test.css"))

rgb(0 255 0);


In [150]:
print(run("advanced.css"))

/* https://www.w3.org/TR/css-color-4/#hex-notation */

.six-digits {
    color: rgb(0 255 0);
}

.eight-digits.upper-case {
    color: rgba(0 0 255 / 0.75294);
}

.three-digits {
    color: rgb(17 34 51);
}

.four-digits {
    color: rgba(0 0 255 / 0.53333);
}



In [152]:
import difflib

got = run("advanced.css")
with open("advanced_expected.css") as f:
    want = f.read()

diff = difflib.unified_diff(got.splitlines(), want.splitlines())
print('\n'.join(diff))